### An Introduction to Value At Risk

#### Introduction

Let $X$ be a random variable describing our financial position,
so that higher values of $X$ represents outcomes convenient to us 
(for instance the value of a stock we own).
We are interested in modeling the *risk* associated to $X$.
Traditionally, a quick way to describe this uncertainty is given by
the standard variation $\sigma(X)$, because it determines how likely it is
to have outcomes far from the expected value (Chebyshev inequality).
But when dealing with risk, this notion is not enough: we are indeed far more
interested in unexpected losses, unfavorable outcomes, while the variance
is intrinsecally symmetric (essentially due to its squared difference
in the definition).
Therefore the need of developing further mathematical methods.




#### Monetary Risk Measures

Let now $\rho :  \mathbb{R} \to \mathbb{R}$ be a generic function.
We want to define it in a way that $\rho(X)$ is capable of capturing
the notion of "risk of the position $X$", conventionally in a way that
lower values are preferred ("risk minimization").
The literature is vast and extends to dozens of alternative definitions,
each with pros and cons, as for instance extensively described in the
book *Stochastic Finance* by Follmer.
To analyze them would go far beyond our purposes, instead we will focus on some
key mathematical propereties before moving to the simplest
"realistic" case in the next section (*Value at Risk*).

Independently on the specific choice of $\rho$, the following property
is established as a requirement:
 - *cash-invariance*: $\rho(X + m) = \rho(X) - m$ for each $m \in \mathbb{R}$;

Cash invariance is interpreted as the fact that if we had more money $m$ at
disposal when getting a position $X$, the overall risk would reduce since it
can be used as a way to countereffect possible negative outcomes, leading to
better overall final position (e.g. avoiding bankrupt).

Any position $Y$ is called *acceptable* if and only if $\rho(Y) \leq 0$.
Observing that $\rho(Y + \rho(Y)) = 0$ we deduce that
any monetary risk measure $\rho(X)$ can be *interpreted* as the minimal amount 
of money needed to make a position $X$ acceptable.

The concrete definition of $\rho$ would change from case to case,
but the overall philosophy remains the same.


#### Quantiles
Let $X$ be a real random variable.
The comulative distribution function of $X$ is defined as:
$F: \mathbb{R} \to [0,1]$, such that $F(t) = P[X \leq t ]$.
The function $F$ is always increasing, but for simplicity we assume to
be continuous and strictly monotonic increasing.
These assumptions allow to quickly define
the *quantile* function $Q: [0,1] \to \mathbb{R}$ simply as $F^{-1}$
without any further remarks: $Q(\lambda) = t$ 
where $t$ is the only value such that
$P[X \leq t] = \lambda$.

In case of discontinuous or not strictly increasing cdf (happening for instance
when working with discrete random variables), quantiles can still be
defined, but the mathematical details need to check
each property go beyond the purpose of this notebook.

Quantile represents a very strong tool to study probability distribution,
and for instance there are statistical tests based on them (Q-Q plots,...).


ADD REMARKS ON COMPUTING QUANTILES AS WELL AS CODE EXAMPLE





#### Value At Risk

Given a financial position $X$ and a *confidence level* $\lambda \in [0, 1]$,
the Value At Risk is defined on the base of quantiles:
$VaR_{\lambda}(V) = Q_{1 - \lambda} (-V)$

The following proposition helps in interpreting the definition:

Proposition: $VaR_{\lambda}(V) = m$ such that $P[V + m < 0] \leq \lambda $


Therefore the VaR can be interpreted as the amount of money that, when added
to the position $X$, make the probability of an overall loss lower than a
selected safety level. Note that in this notation, we want $\lambda$ to be
"small enough". Other resources uses the letter $c = 1 - \lambda$ calling it
the "confidence level". We hope that the context would clarify the chosen
definitions in order to avoid any ambiguity.

Finally by setting $\rho(X) = VaR(X)$ the cash-inavriance property can be
proved making the Value At Risk a monetary risk measure.

Just to clarify, one wants to:
 - choose lambda very small, to stay "safer";
 - minimize the values at risk


HERE SOME CODE TO COMPUTE THE VAR BASED ON THE QUANTILE INTERPRETATION